In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext cython
%timeit

import numpy as np
import cython
import scipy
from scipy import signal
import numpy as np
import os
import subprocess

import matplotlib
from matplotlib import pyplot as plt

TIMEIT = False

### what to optimize

The `benchmarks.txt` we can see that the 3 most costly lines
in the fit function of ThetaScan are:

- seasonal_decompose (79.8 %)
- compute_ses (12.6 %)
- compute_trend (6.2 %)

Therefore those are the functions were most benefit will be achieved.

## `seasonal_decompose` (79.8 %)

The worst part of this function happens in the following function calls:

- `period_averages = np.array([np.nanmean(detrended[i::period], axis=0) for i in range(period)])`

- `trend = convolution_filter(y, period)`

In [2]:
np.random.seed(123)
detrended = np.random.random(20)
period = 5
period_averages = np.array([np.nanmean(detrended[i::period], axis=0) for i in range(period)])
period_averages

array([0.55018727, 0.54461124, 0.3814263 , 0.40586899, 0.51036458])

In [3]:
# the values in period_averages are computed as the mean over values
# that are period positions away starting from 0, 1, 2,...period-1

print(np.nanmean(detrended[0::period]))
print(np.nanmean(detrended[1::period]))
print(np.nanmean(detrended[2::period]))
print(np.nanmean(detrended[3::period]))
print(np.nanmean(detrended[4::period]))

0.5501872669013069
0.5446112427931341
0.38142629824404584
0.40586898525466475
0.5103645826017528


In [20]:
indices = [0,5,10,15]
values = np.array([detrended[j] for j in indices])
print(values)
print(detrended[0::period],'\n')

indices = [1,6,11,16]
values = np.array([detrended[j] for j in indices])
print(values)
print(detrended[1::period])

[0.69646919 0.42310646 0.34317802 0.73799541]
[0.69646919 0.42310646 0.34317802 0.73799541] 

[0.28613933 0.9807642  0.72904971 0.18249173]
[0.28613933 0.9807642  0.72904971 0.18249173]


In [21]:
np.array([np.nanmean(detrended[i::period], axis=0) for i in range(period)])

array([0.55018727, 0.54461124, 0.3814263 , 0.40586899, 0.51036458])

We want to iterate over the data only once tom compute the value in `period_averages`.


Note that `period_averages` is an array of dimension equal to period.

Thefore we can start with an array of lenght period that we will fill while we scan the data. 

In [22]:
period_averages

array([0.55018727, 0.54461124, 0.3814263 , 0.40586899, 0.51036458])

In [75]:
def compute_period_averages(array, period):
    acum_sum = np.zeros(period)
    acum_count = np.zeros(period)
    
    for k in range(len(array)):
        counter_pos = k%period
        acum_sum[counter_pos] += array[k]
        acum_count[counter_pos] +=1
        
    return acum_sum/ acum_count

In [76]:
period = 5

y = [7,8,9,10,11,12,13]
#   [X,0,0,0 ,0 ,X, 0]  -> [7, 12]
#   [0,X,0,0 ,0 ,0, X]  -> [8, 13]
#   [0,0,X,0 ,0 ,0, X]  -> [9]

print(y[0::period], y[1::period], y[2::period])
print(np.mean(y[0::period]), np.mean(y[1::period]), np.mean(y[2::period]))

[7, 12] [8, 13] [9]
9.5 10.5 9.0


In [77]:
compute_period_averages(y, period)

array([ 9.5, 10.5,  9. , 10. , 11. ])

We can see that **`compute_period_averages`** returns the same as our costly function

In [78]:
np.array([np.nanmean(y[i::period], axis=0) for i in range(period)])

array([ 9.5, 10.5,  9. , 10. , 11. ])

In [79]:
def original_period_averages(y, period):
    return np.array([np.nanmean(y[i::period], axis=0) for i in range(period)])

In [80]:
np.testing.assert_almost_equal(original_period_averages(y, period),
                               compute_period_averages(y, period))

In [81]:
if TIMEIT:
    %timeit np.array([np.nanmean(y[i::period], axis=0) for i in range(period)])
    %timeit compute_period_averages(y, period)

Let us try the function with an array that is much bigger


In [82]:
np.random.seed(123)
y = np.random.rand(1000)

In [83]:
compute_period_averages(y, period)

array([0.47432589, 0.49338989, 0.51265333, 0.52439152, 0.49210452])

In [84]:
np.array([np.nanmean(y[i::period], axis=0) for i in range(period)])

array([0.47432589, 0.49338989, 0.51265333, 0.52439152, 0.49210452])

In [85]:
if TIMEIT:
    %timeit original_period_averages(y,period)
    %timeit compute_period_averages(y, period)

Now we see that our non compiled function is slower.

Therefore, our function is not efficient for both small and big inputs.

Let us compile the function to achieve higher performance.

In [86]:
%%cython -a
cimport numpy as cnp
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(False) 
def compute_period_averages_cy(cython.floating[:] x, int period):
    cdef cython.floating[:] p_sum = np.zeros(period)
    cdef cython.floating[:] p_count = np.zeros(period)
    cdef int k, counter_pos
    cdef int n_x = len(x)
    
    for k in range(n_x):
        counter_pos = k % period
        p_sum[counter_pos] += x[k]
        p_count[counter_pos] += 1
    
    return np.array(p_sum)/ np.array(p_count)

In [147]:
np.testing.assert_almost_equal(original_period_averages(y, period),
                               compute_period_averages_cy(y, period))

In [92]:
if TIMEIT:
    %timeit original_period_averages(y, period)
    %timeit compute_period_averages_cy(y, period)

In [4]:
%%cython -a
cimport numpy as cnp
import numpy as np
cimport cython
from libc.math cimport isnan

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True) 
def compute_period_averages_cy2(cython.floating[:] array, int period):
    cdef cython.floating[:] acum_sum = np.zeros(period)
    cdef cython.floating[:] acum_count = np.zeros(period)
    cdef int k, counter_pos
    cdef cython.floating x_k
    
    for k in range(len(array)):
        counter_pos = k % period
        
        x_k = array[k]
        if not isnan(x_k):
            acum_sum[counter_pos] += x_k
            acum_count[counter_pos] += 1

    for k in range(period):
        acum_sum[k]/= acum_count[k]
        
    return np.array(acum_sum)

In [137]:
np.testing.assert_almost_equal(original_period_averages(y, period),
                               compute_period_averages_cy2(y, period))

In [142]:
if True:
    %timeit original_period_averages(y, period)
    %timeit compute_period_averages_cy2(y, period)

127 µs ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.27 µs ± 97.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
import copy

y_cop = copy.deepcopy(y)
y_cop[0] = np.nan

np.testing.assert_almost_equal(original_period_averages(y_cop, period),
                               compute_period_averages_cy2(y_cop, period))


NameError: name 'y' is not defined

###  `compute_ses`

In [9]:

def compute_ses(y, alpha):
    nobs = len(y)  

    # Forecast Array
    fh = np.full(nobs + 1, np.nan)  
    fh[0] = y[0]  
    fh[1] = y[0]  

    # Simple Exponential Smoothing
    for t in range(2, nobs + 1):
        fh[t] = alpha * y[t - 1] + (1 - alpha) * fh[t - 1]  # s[t] = alpha * y....

    return (fh[:nobs], fh[nobs])

In [13]:
%%cython -a
cimport numpy as cnp
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef compute_ses_cy(cython.floating[:] y, cython.floating alpha):
    cdef int n_obs = len(y)  
    cdef int t
    cdef cython.floating[:] fh = np.empty(n_obs + 1)
    cdef cython.floating alpha_y, alpha_fh

    fh[0] = y[0]
    fh[1] = y[0]
    
    for t in range(2, n_obs + 1):
        fh[t] = alpha * y[t - 1] + (1 - alpha) * fh[t - 1]
        
    return (fh[:n_obs], fh[n_obs])

In [11]:
np.random.seed(123)
y = np.random.random(1000)

In [12]:
if True:
    time_compute_ses = %timeit -o compute_ses(y, 0.05)
    time_compute_ses_cy = %timeit -o compute_ses_cy(y, 0.05)
    res = time_compute_ses.average / time_compute_ses_cy.average
    print(f'time improvement compute_ses cython: {res} X')

571 µs ± 9.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.99 µs ± 57.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
time improvement compute_ses cython: 81.6314140666947 X


In [ ]:
compute_ses_cy(y, 0.05)[1] == compute_ses(y, 0.05)[1]

### `convolution_filter`

In [18]:

def convolution_filter(y, period):
    # Prepare Filter
    if period % 2 == 0:
        filt = np.array([.5] + [1] * (period - 1) + [.5]) / period
    else:
        filt = np.repeat(1. / period, period)

    # Signal Convolution
    conv_signal = scipy.signal.convolve(y, filt, mode='valid')

    # Padding (2-Sided Convolution)
    trim_head = int(np.ceil(len(filt) / 2.) - 1) or None
    trim_tail = int(np.ceil(len(filt) / 2.) - len(filt) % 2) or None

    if trim_head:
        conv_signal = np.r_[conv_signal, [np.nan] * trim_tail]
    if trim_tail:
        conv_signal = np.r_[[np.nan] * trim_head, conv_signal]

    return conv_signal

In [19]:
if TIMEIT:
    %timeit res = convolution_filter(y,4)

In [25]:
period = 4

def get_filt(period):

    if period % 2 == 0:
        filt = np.array([.5] + [1] * (period - 1) + [.5]) / period
    else:
        filt = np.repeat(1. / period, period)
    
    return filt

print(get_filt(2), f'len={len(get_filt(2))}')
print(get_filt(3), f'len={len(get_filt(3))}')
print(get_filt(4), f'len={len(get_filt(4))}')
print(get_filt(5), f'len={len(get_filt(5))}')
print(get_filt(6), f'len={len(get_filt(6))}')

[0.25 0.5  0.25] len=3
[0.33333333 0.33333333 0.33333333] len=3
[0.125 0.25  0.25  0.25  0.125] len=5
[0.2 0.2 0.2 0.2 0.2] len=5
[0.08333333 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667
 0.08333333] len=7


 If the len of the filter is
 
- odd, then filter[0] and filter[-1] are halve the value that is not in the extremes

- even, then all values in filter are the same

